In [1]:
import numpy as np
from scipy.linalg import block_diag

# Number of sensors
M = 12

# Number of nodes
K = 6

# Locations of the nodes with respect to initial node
locations = (np.array([0,0]),np.array([0.45,0.99]),np.array([3.02,0.45]),np.array([5.61,0.90])
             ,np.array([8.03,1.46]),np.array([8.70,0.50]))

# Direction of arrivals
angles = np.array([-14,-10,5])*2*(np.pi)/360
K_l = np.array([np.sin(angles), np.cos(angles)])

# Steering vector is implemented. It should have the shape: (12,3)
for j in range(0,len(locations)):
    for i in range(0,len(angles)):
        first_part = np.exp(1j*np.pi*locations[j].dot(K_l[:,i]))
        second_part = np.array([1, np.exp(1j*np.pi*1*np.sin(angles[i]))]) 
        A_k = np.transpose(np.array(first_part*second_part, ndmin=2))
        if i == 0:
            B_k = A_k
        else:
            B_k = np.concatenate((B_k, A_k), axis=1)
        
        if i == len(angles)-1:
            if j == 0:
                A = np.transpose(B_k)
            else:
                A = np.concatenate((A, np.transpose(B_k)), axis=1)

A = np.transpose(A)

# Signal(A*s) to noise(n) ratio 
ratio_As_to_s = 0.23866
snr = 10*ratio_As_to_s
N_slot = 500

# Source signal implementation (shape: (3,500))
signal = np.random.normal(0,np.sqrt(snr),(3,N_slot)) + 1j*np.random.normal(0,np.sqrt(snr),(3,N_slot))

# Received signal power on sensors
signal_power = np.var(A.dot(signal))

# Noise signal implementation (shape: (12,500))
noise = np.random.normal(0,np.sqrt(1),(12,N_slot)) + 1j*np.random.normal(0,np.sqrt(1),(12,N_slot))

print("SIGNAL TO NOISE RATIO")
print(signal_power/np.sqrt(np.var(noise)))

# Received signal (shape: (12,500))
z = A.dot(signal) + noise

# Sample covariance matrix
R_sample = z.dot(z.conj().T)

# Eigenvalue and eigenvectors
w_sample, v_sample = np.linalg.eig(R_sample)
print()
print("EIGENVALUES OF SAMPLE COVARIANCE MATRIX")
print(w_sample[0])
print(w_sample[1])
print(w_sample[2])
print(w_sample[3])

# Sensor Selection Matrix (shape: (12,6))
T = np.array([[1,0,0,0,0,0],
              [1,0,0,0,0,0],
              [0,1,0,0,0,0],
              [0,1,0,0,0,0],
              [0,0,1,0,0,0],
              [0,0,1,0,0,0],
              [0,0,0,1,0,0],
              [0,0,0,1,0,0],
              [0,0,0,0,1,0],
              [0,0,0,0,1,0],
              [0,0,0,0,0,1],
              [0,0,0,0,0,1]])

# Push-Sum Matrix (shape: (6,6))
P = np.array([[0.3,0.15,0.25,0.14,0.22,0.2],
              [0.2,0.15,0.25,0.23,0.22,0.2],
              [0.1,0.15,0.2,0.27,0.06,0.3],
              [0.1,0.15,0.2,0.12,0.35,0.15],
              [0.1,0.2,0.05,0.14,0.05,0.10],
              [0.2,0.2,0.05,0.1,0.1,0.05]]) 

# Weight Vector  (shape: (6,1))
w = np.atleast_2d([1,1,1,1,1,1]).T

# Push-Sum Covariance Matrix Estimation
R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P,10)).dot(T.T), R_sample)
R_push_denominator = T.dot(np.linalg.matrix_power(P,10)).dot(w).dot(np.ones((1,6))).dot(T.T)

# Push Sum Covariance Matrix (shape: (12,12))
R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
print()
print("                            R_push")
print("****************************************************************")
print("****************************************************************")
print(R_push)
print()
print("                           R_sample")
print("****************************************************************")
print("****************************************************************")
print(R_sample)
print()
# Check Convergence Status
print(R_push == R_sample)

w_push, v_push = np.linalg.eig(R_push)

# Upper group selection matrix J_up
J_up = np.kron(np.eye(6),np.array([1,0]))

# Lower group selection matrix J_down
J_down = np.kron(np.eye(6),np.array([0,1]))

# Push-Sum estimated signal eigenvector matrices
U_s_push = v_push[:,:3]

# Upper signal eigenvectors
U_s_up = J_up.dot(U_s_push)

# Lower signal eigenvectors
U_s_down = J_down.dot(U_s_push)

# Matrix including knowledge about DOAs of the source signals
psi = np.linalg.inv((U_s_up.conj().T).dot(U_s_up)).dot((U_s_up.conj().T)).dot(U_s_down)

w2, v2 = np.linalg.eig(psi)
print()
print("           DOAs of the source signals in degrees")
print("****************************************************************")
print("****************************************************************")
doa_1 = np.arcsin(np.angle(w2[0])/np.pi)/(2*np.pi)*360
print("DOA of the first source signal:   " + str(doa_1))
doa_2 = np.arcsin(np.angle(w2[1])/np.pi)/(2*np.pi)*360
print("DOA of the second source signal:   " + str(doa_2))
doa_3 = np.arcsin(np.angle(w2[2])/np.pi)/(2*np.pi)*360
print("DOA of the third source signal:   " + str(doa_3))



SIGNAL TO NOISE RATIO
10.220274509225737

EIGENVALUES OF SAMPLE COVARIANCE MATRIX
(56959.83416581038-2.248576716537617e-12j)
(24816.113977081834+1.2354186039988076e-12j)
(7119.793222698791+1.3513625867490035e-12j)
(1205.2859570297085-4.49041572514913e-13j)

                            R_push
****************************************************************
****************************************************************
[[ 8502.31839774+0.00000000e+00j  6331.29379615+2.11476971e+03j
  -7031.03741391-1.54748645e+03j -4694.19694497-3.23378625e+03j
   2206.06294254+6.19746229e+02j  -717.35140039+2.86836457e+03j
   1644.11868622+5.44650269e+03j   -76.57907367+6.35697604e+03j
   4696.60780541+9.46555116e+02j  2286.84466855+1.01054678e+03j
  -3291.71338188-7.05523114e+02j  -929.78506745-2.72457031e+02j]
 [ 6331.29379615-2.11476971e+03j  8300.5048858 +0.00000000e+00j
  -6973.52675273+7.14054701e+02j -6809.36728214-1.70756354e+03j
   3600.51437726-2.52136158e+03j  2063.85216059+7.04738950e+02j


In [2]:
print(psi.shape)

(3, 3)
